‘(파이토치 트랜스포머를 활용한) 자연어 처리와 컴퓨터비전 심층학습’
범위 : p591-p599 (YOLOv8)


##### YOLOv8 모델로 포즈 추정(pose estimation) 수행

In [ ]:
!pip install ultralytics opencv-python

In [ ]:
#YOLOv8 포즈 추정 모델 불러오기

from ultralytics import YOLO

model = YOLO('yolov8n-pose.pt') #YOLO 클래스를 통해 사전 학습된 yolovim-pose 모델 불러오기


In [ ]:
#비디오 파일을 이용해 실시간 포즈 추정

#비디오 파일 불러오기
import cv2

capture = cv2.VideoCapture("../datasets/woman.mp4")
while cv2.waitKey(10) < 0:
  if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
  ret, frame = capture.read()
  cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

코랩에서는 오류가 나서 아래 코드로 수정

In [ ]:
# 비디오 파일을 이용해 실시간 포즈 추정

# 비디오 파일 불러오기
import cv2
from google.colab.patches import cv2_imshow

capture = cv2.VideoCapture("/content/woman.mp4")

frame_cnt = 0  # 몇 프레임까지 볼지 제어

while True:  #  키보드 입력 대신 무한 루프 + 조건 종료
    # 마지막 프레임까지 가면 다시 처음으로
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    if not ret:   # 프레임을 못 읽으면 종료
        break

    cv2_imshow(frame)  #  cv2.imshow() 대신 사용

    frame_cnt += 1
    if frame_cnt > 200:  # 예시: 200프레임만 보고 종료
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
import cv2, os

path = "/content/woman.mp4"
print(os.path.exists(path))  # True여야 함
capture = cv2.VideoCapture(path)
print("Opened:", capture.isOpened())  # True여야 함


In [ ]:
!apt install -y ffmpeg
!ffmpeg -i /content/woman.mp4 -vcodec libx264 -acodec aac woman_fixed.mp4


In [ ]:
capture = cv2.VideoCapture("woman_fixed.mp4")
print("Opened:", capture.isOpened())


In [ ]:
#모델 추론

import torch

def predict(frame, iou=0.7, conf=0.25):
  results = model(
      source=frame,
      device=0 if torch.cuda.is_available() else 'cpu',
      iou=0.7, #중복된 경계 상자를 제거하는 임곗값
      conf=0.25, #클래스 점수 임곗값 - 이 값보다 낮은 값은 제거
      verbose=False,#로그 정보- 모델 수행 시 출력되는 정보 표시 여부 결정
  )
  result = results[0]
  return result



In [ ]:
#경계 상자 시각화

def draw_boxes(result, frame):
  for boxes in result.boxes:
    x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 1) #이미지, x, y, 색상, 선 두께
  return frame


In [ ]:
#모델 추론 및 시각화 적용

ret, frame = capture.read()
result = predict(frame)
frame = draw_boxes(result, frame)
cv2.imshow("VideoFrame", frame)


In [ ]:
# 모델 추론 및 시각화 적용
import cv2
from google.colab.patches import cv2_imshow

ret, frame = capture.read()

if not ret:
    print("프레임을 읽을 수 없습니다.")
else:
    result = predict(frame)
    frame = draw_boxes(result, frame)
    cv2_imshow(frame)  # Colab에서는 cv2.imshow() 대신 이걸 사용

capture.release()
cv2.destroyAllWindows()


In [ ]:
!pip install ultralytics


In [ ]:
#키 포인트 시각화

#from ultralytics.yolo.plotting import Annotator
from ultralytics.utils.plotting import Annotator  # 최신 버전 경로

def draw_keypoints(result, frame):
  annotator = Annotator(frame, line_width=1)
  for kps in result.keypoints: #키 포인트 kps 는 사람의 수만큼 존재할 수 있다.
    kps = kps.data.squeeze()
    annotator.kpts(kps)
    nkps = kps.cpu().numpy()

    for idx, (x,y,score) in enumerate(nkps):
      if score > 0.5:
        cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), cv2.FILLED)
        cv2.putText(
            frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1
        )
  return frame